In [5]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, TimeDistributed
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, Dropout, Attention
from tensorflow.keras.models import Model
import re
import numpy as np
from tensorflow.keras.models import load_model

import pandas as pd
import glob

In [24]:
import pandas as pd
import re
import glob

file_pattern = f"data/conve*.txt"
matching_files = glob.glob(file_pattern)

input_texts = []
output_texts = []
df = pd.DataFrame()
for file_path in matching_files:
    with open(file_path, "r") as file:
        content = file.read()

    examples = content.split("assistant: ")

    inputs = []
    outputs = []

    i = 1
    for example in examples[1:]:
        for _ in range(100):
            input_match = re.search(f"Input{i}: (.+?)\n", example)
            output_match = re.search(f"Output{i}: (.+?)\n", example)

            if input_match and output_match:
                # outputs_value = "flowchart TD \n\t" + output_match.group(1)
                outputs_value = output_match.group(1)
                # outputs_value = format_mermaid_code(outputs_value)
                inputs_value = re.sub(r'Output.*', r'', input_match.group(1))
                inputs.append(inputs_value)
                outputs.append(outputs_value)
                output_texts.append(outputs_value)
                input_texts.append(inputs_value)
            i += 1  
    newDf = pd.DataFrame({"input": inputs, "output": outputs})
    df = pd.concat([df, newDf], ignore_index=True)


A[Start] --> B[Search the string] B --> C{Most repeated character found?} C -- No --> B C -- Yes --> D[Remove all occurrences] D --> E[End]


In [11]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Attention, Concatenate
from tensorflow.keras.models import Model
import numpy as np

# Define the maximum sequence length for input and output sequences
max_input_seq_len = max(len(seq) for seq in input_texts)
max_output_seq_len = max(len(seq) for seq in output_texts)

# Define the input and output tokenizers
input_tokenizer = Tokenizer(filters='', char_level=False)
output_tokenizer = Tokenizer(filters='', char_level=False)

# Fit the input tokenizer on the preprocessed input sequences
input_tokenizer.fit_on_texts(input_texts)

# Fit the output tokenizer on the preprocessed output sequences
output_tokenizer.fit_on_texts(output_texts)

# Convert the input and output sequences to numerical format
encoder_input_seqs = input_tokenizer.texts_to_sequences(input_texts)
decoder_input_seqs = output_tokenizer.texts_to_sequences(output_texts)
decoder_target_seqs = [seq[1:] for seq in decoder_input_seqs]

# Pad the input and output sequences to fixed length
encoder_input_seqs = pad_sequences(encoder_input_seqs, maxlen=max_input_seq_len, padding='post')
decoder_input_seqs = pad_sequences(decoder_input_seqs, maxlen=max_output_seq_len, padding='post')
decoder_target_seqs = pad_sequences(decoder_target_seqs, maxlen=max_output_seq_len, padding='post')

# Define the model architecture
latent_dim = 256

# Encoder
encoder_inputs = Input(shape=(max_input_seq_len,))
encoder_embedding = Embedding(input_dim=len(input_tokenizer.word_index) + 1, output_dim=latent_dim, mask_zero=True)(encoder_inputs)
encoder_lstm, state_h, state_c = LSTM(latent_dim, return_sequences=True, return_state=True)(encoder_embedding)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(max_output_seq_len,))
decoder_embedding = Embedding(input_dim=len(output_tokenizer.word_index) + 1, output_dim=latent_dim, mask_zero=True)(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)

# Attention layer
attention = Attention()([decoder_outputs, encoder_lstm])
context = Concatenate(axis=-1)([attention, decoder_outputs])

# Dense layer for prediction
decoder_dense = Dense(len(output_tokenizer.word_index) + 1, activation='softmax')
decoder_outputs = decoder_dense(context)

# Define the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit([encoder_input_seqs, decoder_input_seqs], decoder_target_seqs, epochs=10, batch_size=1)

# Save the model
model.save('my_model4.h5')


Epoch 1/10


2023-11-19 00:17:08.112468: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:693] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "CPU" vendor: "GenuineIntel" model: "110" frequency: 1800 num_cores: 8 environment { key: "cpu_instruction_set" value: "AVX SSE, SSE2, SSE3, SSSE3, SSE4.1, SSE4.2" } environment { key: "eigen" value: "3.4.90" } l1_cache_size: 32768 l2_cache_size: 262144 l3_cache_size: 6291456 memory_size: 268435456 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }


2293/2293 [==============================] - 3314s 1s/step - loss: 4.0419 - accuracy: 0.3982
Epoch 2/10
2293/2293 [==============================] - 3232s 1s/step - loss: 2.7156 - accuracy: 0.5170
Epoch 3/10
2293/2293 [==============================] - 3190s 1s/step - loss: 2.0679 - accuracy: 0.5756
Epoch 4/10
2293/2293 [==============================] - 3179s 1s/step - loss: 1.5552 - accuracy: 0.6305
Epoch 5/10
2293/2293 [==============================] - 3183s 1s/step - loss: 1.1518 - accuracy: 0.6904
Epoch 6/10
2293/2293 [==============================] - 3184s 1s/step - loss: 0.8397 - accuracy: 0.7575
Epoch 7/10
2293/2293 [==============================] - 3190s 1s/step - loss: 0.6248 - accuracy: 0.8116
Epoch 8/10
2293/2293 [==============================] - 3186s 1s/step - loss: 0.4748 - accuracy: 0.8543
Epoch 9/10
2293/2293 [==============================] - 3188s 1s/step - loss: 0.3558 - accuracy: 0.8904
Epoch 10/10
2293/2293 [==============================] - 3186s 1s/step - lo

In [16]:

# Load the trained model
model = load_model('my_model4.h5')

# Define the input sentence
input_sentence = "Scan a list of prices until finding the one with the highest value"

# Preprocess the input sentence
input_sequence = input_tokenizer.texts_to_sequences([input_sentence])
padded_input_sequence = pad_sequences(input_sequence, maxlen=max_input_seq_len, padding='post')

# Initialize the decoded sequence
decoded_sequence = np.zeros((1, max_output_seq_len))

# Use the model to predict the output sequence
for i in range(max_output_seq_len):
    predictions = model.predict([padded_input_sequence, decoded_sequence])
    predicted_token_index = np.argmax(predictions[0, i, :])
    decoded_sequence[0, i] = predicted_token_index

    if predicted_token_index == 0:  # End of sequence token
        break

# Convert the predicted sequence back to text
output_sentence = output_tokenizer.sequences_to_texts([decoded_sequence[0]])

print("Input sentence:", input_sentence)
print("Predicted output:", output_sentence)


2023-11-19 10:05:01.859640: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:693] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "CPU" vendor: "GenuineIntel" model: "110" frequency: 1800 num_cores: 8 environment { key: "cpu_instruction_set" value: "AVX SSE, SSE2, SSE3, SSSE3, SSE4.1, SSE4.2" } environment { key: "eigen" value: "3.4.90" } l1_cache_size: 32768 l2_cache_size: 262144 l3_cache_size: 6291456 memory_size: 268435456 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }


1/1 [==============================] - 0s 181ms/step
Input sentence: Scan a list of prices until finding the one with the highest value
Predicted output: ['the first name] b --> c[choose the desired number] c --> d[iterate through the words] d --> e[apply toothpaste to the desired location] e --> f[add the timer] f --> d d -- no --> g[end]']
